In [13]:
import re

# Input text
text = """
<day_1_date> : 2021-05-02. <day_1_weather_forecast> : Heat is expected to build in response to the upper level ridge, with highs 10-20 degrees above normal possible Wednesday-Friday.. This will equate to some 100+ degree high temperatures for the lower elevations of eastern California and southern Arizona.. Seasonal to below normal temperatures are expected for the weekend after the passage of the cold front on Saturday.. Heavy showers and storms remain likely from the central Gulf Coast into the southeast Wednesday into Thursday.. Unsettled coastal conditions to include some moderate rains for the Mid-Atlantic and Northeast due to a developing coastal low.. Potential for some moderate precipitation from the Pacific Northwest to the Northern Rockies where terrain enhanced snows are possible.. . . A weather pattern expected to significantly amplify with upper troughs working into the West and East, and a warming ridge in the west-central U.S.. Confidence in the forecast is bolstered by above average agreement among models and ensembles.. Major rainfall threats and temperature extremes described, indicating a dynamic weather week. <day_1_temp> : '68.3'.
<day_2_date> : 2021-05-03. <day_2_weather_forecast> : Cooler conditions are expected in the wake of the cold front.. <day_2_temp> : '55.4'.
"""

# Extract all temperatures using regex
temperature_matches = re.findall(r"<day_\d+_temp> : '([\d.]+)'", text)

# Format the extracted temperatures
formatted_temperatures = [[float(temp)] for temp in temperature_matches]

# Remove the number part in the text (like <day_1_temp>)
modified_text = re.sub(r"<day_\d+_temp> : '([\d.]+)'", "", text)

print("Formatted Temperatures:", formatted_temperatures)
print("\nModified Text:\n", modified_text)


Formatted Temperatures: [[68.3], [55.4]]

Modified Text:
 
<day_1_date> : 2021-05-02. <day_1_weather_forecast> : Heat is expected to build in response to the upper level ridge, with highs 10-20 degrees above normal possible Wednesday-Friday.. This will equate to some 100+ degree high temperatures for the lower elevations of eastern California and southern Arizona.. Seasonal to below normal temperatures are expected for the weekend after the passage of the cold front on Saturday.. Heavy showers and storms remain likely from the central Gulf Coast into the southeast Wednesday into Thursday.. Unsettled coastal conditions to include some moderate rains for the Mid-Atlantic and Northeast due to a developing coastal low.. Potential for some moderate precipitation from the Pacific Northwest to the Northern Rockies where terrain enhanced snows are possible.. . . A weather pattern expected to significantly amplify with upper troughs working into the West and East, and a warming ridge in the wes